# Milestone 3: Data Exploration

#### Evan Brown, Felix Ruano, Evan Brown

## EVAN: digging into data regarding inspection outcomes. Yelp API

### Yelp

In [3]:
from yelp.client import Client
from yelp.oauth1_authenticator import Oauth1Authenticator

auth = Oauth1Authenticator(
    consumer_key= 'Oy7XguSi2LNqkezRy9FpLg',
    consumer_secret= 'vCd553h9VA481BjS2aoy_zy5uwM',
    token= 'A1038kcxEgES1ARXf8xpAWHr8y_MUaWo',
    token_secret= 'rqXou8gBHNWYkUcE1_n2d3JAihI'
)

client = Client(auth)

In [4]:
type(client)

yelp.client.Client

In [18]:
params = {
    'term': 'food',
    'lang': 'eng'
}

results = client.search('Chicago', **params)

In [21]:
results.businesses[0].name

u'The Purple Pig'

## GIL: writing up data exploration into number of violations/ data cleaning. + weather temporal data

## FELIX: Geographical - Lat/Long etc. COLLATE INTO ONE NOTEBOOK

### Also to look at:

* Facility type
* Zip code
* **Inspection Type**